In [ ]:
from reggae.models import TranscriptionLikelihood, Options
from reggae.data_loaders import load_barenco_puma, load_3day_dros, DataHolder, scaled_barenco_data
from reggae.utilities import get_rbf_dist, discretise, logit, logistic
from scipy.interpolate import interp1d
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import math as tfm
import numpy as np
from matplotlib import pyplot as plt

np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
plt.style.use('ggplot')
%matplotlib inline
f64 = np.float64


In [ ]:
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()

# m_observed, f_observed, t = load_3day_dros()

replicate = 0

m_df, m_observed = m_observed 
f_df, f_observed = f_observed

# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed[replicate]
f_observed = np.atleast_2d(f_observed[replicate])
σ2_m_pre = σ2_m_pre[0]
σ2_f_pre = σ2_f_pre[0]

num_genes = m_observed.shape[0]
t = np.arange(10)
τ, common_indices = discretise(t)

N_p = τ.shape[0]
N_m = m_observed.shape[1]

f_observed = tf.stack([f_observed[0], f_observed[0], f_observed[0]])

data = (m_observed, f_observed)
noise_data = (σ2_m_pre, σ2_f_pre)
time = (t, τ, (common_indices))

data = DataHolder(data, noise_data, time)
N_p = τ.shape[0]

opt = Options(preprocessing_variance=True, tf_mrna_present=True)
lik = TranscriptionLikelihood(data, opt)
num_tfs = 3

In [ ]:
# Transcription factor

A = np.array([0.01, 0.3, 0.47, 0.51, 0.4, 0.37, 0.47, 0.32, 0.16, 0.025])
B = np.array([0.01, 0.1, 0.22, 0.44, 0.53, 0.41, 0.23, 0.13, 0.05, 0.013])
C = np.array([0.01, 0.02, 0.03, 0.05, 0.08, 0.16, 0.4, 0.35, 0.23, 0.02])
interp = interp1d(np.arange(A.shape[0]), A, kind='cubic')
A = interp(np.linspace(0,9, τ.shape[0]))
interp = interp1d(np.arange(B.shape[0]), B, kind='cubic')
B = interp(np.linspace(0,9, τ.shape[0]))
interp = interp1d(np.arange(C.shape[0]), C, kind='cubic')
C = interp(np.linspace(0,9, τ.shape[0]))

δbar = logistic(f64(np.array([1.5, 1.5, 1.5])))

fbar = np.array([A, B, C])
fbar = 5*preprocessing.normalize(fbar)
print(fbar.shape)
tf_labels = ['A', 'B', 'C']
plt.title('TFs')
for i in range(num_tfs):
    plt.plot(np.arange(τ.shape[0]), fbar[i], label=f'TF {i}')
plt.legend()

fbar = tfm.log((tfm.exp(fbar)-1))

plt.figure()
plt.title('TF Proteins')

f_i = tfm.log(1+tfm.exp(fbar))
f = tfm.log(1+tfm.exp(fbar[0]))

p = lik.calculate_protein(fbar, δbar)

for i in range(num_tfs):
    plt.plot(p[i], label=f'Protein {i}')
plt.legend()

In [ ]:
# Kinetic parameters
#   [[initial, base, sens, decay]]


In [ ]:
w = 1*tf.ones((num_genes, num_tfs), dtype='float64') # TODO
w_0 = tf.zeros(num_genes, dtype='float64') # TODO
kbar = logistic(np.array([[0.2061, 0.2475, 0.8222, 4.0416],
                       [0.4091, 0.7305, 0.9486, 2.2348],
                       [0.1304, 0.3921, 2.3116, 7.1835],
                       [0.3789, 0.2861, 1.2456, 0.9928],
                       [0.2906, 0.6604, 0.8742, 4.1688]]))
print(kbar)
m_pred = lik.predict_m(kbar, δbar, w, fbar, w_0)
# m_pred2 = lik2.predict_m(kbar, δbar[0], w, fbar[0], w_0)

print(m_pred.shape)
plt.figure(figsize=(14, 10))
for j in range(num_genes):
    plt.subplot(num_genes*100+21+j)
    plt.title(f'Gene {j}')
    plt.plot(m_pred[j], color='grey')
    
plt.tight_layout()
# plt.figure(figsize=((10, 5)))
# for j in range(num_genes):
#     plt.plot(m_pred2[j], label=j)
